In [2]:
# Standard packages first, then others in alphabetical order

import subprocess
from io import BytesIO

import folium
import hvplot.pandas
import pandas as pd
import requests


In [3]:
%%capture
%%bash
jupyter nbconvert time-series-flooding.ipynb --to html --no-input